In [1]:
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

     ------------------------------------ 342.5/342.5 MB 793.5 kB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\users\\dell\\appdata\\local\\programs\\python\\python37\\lib\\site-packages\\~-nsorflow\\lite\\experimental\\microfrontend\\python\\ops\\_audio_microfrontend_op.so'
Consider using the `--user` option or check the permissions.



In [1]:
import gym 
import random

In [2]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [3]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [4]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\gym\envs\atari\environment.py:269: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  "We strongly suggest supplying `render_mode` when "
c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\pyglet\image\codecs\wic.py:406: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


Episode:1 Score:245.0
Episode:2 Score:460.0
Episode:3 Score:145.0
Episode:4 Score:400.0
Episode:5 Score:85.0


In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [6]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [12]:
del model

In [26]:
model = build_model(height, width, channels, actions)

In [27]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_3 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 512)               34603520  
_________________________________________________________________
dense_13 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_14 (Dense)             (None, 6)                

In [9]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [10]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [28]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

In [15]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
  619/10000: episode: 1, duration: 33.199s, episode steps: 619, steps per second:  19, episode reward: 150.000, mean reward:  0.242 [ 0.000, 25.000], mean action: 2.538 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 1208/10000: episode: 2, duration: 533.142s, episode steps: 589, steps per second:   1, episode reward: 20.000, mean reward:  0.034 [ 0.000, 15.000], mean action: 2.362 [0.000, 5.000],  loss: 11.995610, mean_q: 3.218847, mean_eps: 0.900640
 2016/10000: episode: 3, duration: 2095.321s, episode steps: 808, steps per second:   0, episode reward: 110.000, mean reward:  0.136 [ 0.000, 30.000], mean action: 2.479 [0.000, 5.000],  loss: 0.672121, mean_q: 1.690915, mean_eps: 0.854965
 2695/10000: episode: 4, duration: 1601.611s, episode steps: 679, steps per second:   0, episode reward: 225.000, mean reward:  0.331 [ 0.000, 30.000], 

In [16]:
dqn.save_weights('SavedWeights/10k-Fast/dqn_weights.h5f')

In [31]:

scores = dqn.test(env, nb_episodes=5, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 5 episodes ...
Episode 1: reward: 405.000, steps: 993
Episode 2: reward: 80.000, steps: 667
Episode 3: reward: 110.000, steps: 724
Episode 4: reward: 65.000, steps: 607
Episode 5: reward: 70.000, steps: 526
146.0


In [18]:
env.close()

In [25]:
del model, dqn

In [30]:
dqn.load_weights('SavedWeights/10k-Fast/dqn_weights.h5f')